In [2]:
from langchain.llms import OpenAI
%pip install python-dotenv

In [3]:
from dotenv import load_dotenv #to access environment variables in .env file
import os

load_dotenv()



True

In [4]:
%pip install --upgrade --quiet  langchain-openai


# Import Azure OpenAI
from langchain_openai import AzureChatOpenAI

Note: you may need to restart the kernel to use updated packages.


We can use both os.getenv and os.environ, just make sure you are using right format


for os.getenv("")

for os.environ[""]

In [5]:
# check the os environment variable settings
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

llm = AzureChatOpenAI(
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment="gpt-35-turbo",
)

text="Write a poem about India"

print(llm.invoke(text))

content="India, the land of vibrant colors and sounds,\nWhere history and modernity astound,\nA land of diverse cultures and traditions,\nOf timeless tales and mystic visions.\n\nFrom the snow-capped Himalayan peaks,\nTo the sandy shores of the southern seas,\nIndia's beauty is beyond compare,\nA land of wonders, oh so rare.\n\nThe Taj Mahal's white marble dome,\nA symbol of love, forever known,\nThe Red Fort's grandeur, a sight to see,\nA testimony to India's history.\n\nThe Ganges, sacred and divine,\nA river that flows through space and time,\nA land of spirituality and faith,\nWhere gods and goddesses hold sway.\n\nThe bustling cities, alive with energy,\nThe markets and bazaars, a cacophony,\nOf fragrances, flavors, and sights,\nA sensory overload, day and night.\n\nIndia, a country that captures the heart,\nOf those who visit, and never depart,\nA land of warmth and hospitality,\nWhere every visitor is treated royally.\n\nIndia, a jewel in the crown,\nOf the world's cultural heri

### Prompt Templates And LLMChain


In [6]:
from langchain.prompts import PromptTemplate

prompt_template= PromptTemplate.from_template('what is famous in {state}')

prompt_template.format(state='Tamil Nadu')

'what is famous in Tamil Nadu'

In [7]:
from langchain.chains import LLMChain

chain= LLMChain(llm=llm, prompt=prompt_template)
chain.invoke("Tamil Nadu")


{'state': 'Tamil Nadu',
 'text': 'Tamil Nadu is famous for its rich cultural heritage, art, literature, dance forms, music, cuisine, and historical monuments. Some of the famous attractions in Tamil Nadu are:\n\n1. Meenakshi Temple in Madurai\n2. Brihadeeswara Temple in Thanjavur\n3. Mahabalipuram Shore Temple\n4. Marina Beach in Chennai\n5. Pongal Festival\n6. Chettinad Cuisine\n7. Bharatanatyam Dance\n8. Kanchipuram Silk Sarees\n9. Tamil Literature and Language\n10. Hill Stations like Ooty and Kodaikanal.'}

### Combining Multiple Chains Uing simple Sequential Chain


In [8]:
prompt_template1= PromptTemplate.from_template('what is the capital of {country}')
chain1= LLMChain(llm=llm, prompt=prompt_template1)
prompt_template2= PromptTemplate.from_template('what is famous in {capital}')
chain2= LLMChain(llm=llm, prompt=prompt_template2)



In [9]:
from langchain.chains import SimpleSequentialChain
schain=SimpleSequentialChain(chains=[chain1,chain2])
schain.invoke("Singapore")

{'input': 'Singapore',
 'output': 'While this statement is true, it is not informative. Here are some famous attractions in Singapore that people may be interested in visiting:\n\n- Marina Bay Sands\n- Gardens by the Bay\n- Sentosa Island\n- Merlion Park\n- Orchard Road\n- Clarke Quay\n- Chinatown\n- Universal Studios Singapore\n- Singapore Zoo\n- National Gallery Singapore\n- Hawker centres for local food\n\nSingapore is also known for its cleanliness, efficiency, and safety.'}

### Sequential Chain


In [10]:
from langchain.chains import SequentialChain


prompt_template1= PromptTemplate.from_template('what is the capital of {country}')
chain1= LLMChain(llm=llm, prompt=prompt_template1,output_key='capital')
prompt_template2= PromptTemplate.from_template('what is famous in {capital}')
chain2= LLMChain(llm=llm, prompt=prompt_template2,output_key='places')



sschain=SequentialChain(chains=[chain1,chain2],
                        input_variables=["country"],
                        output_variables=["capital","places"])

sschain.invoke("Malaysia")


{'country': 'Malaysia',
 'capital': 'The capital of Malaysia is Kuala Lumpur.',
 'places': 'Kuala Lumpur is famous for its iconic landmarks such as the Petronas Twin Towers, Batu Caves, Menara Kuala Lumpur, Bukit Bintang shopping district, and the Kuala Lumpur Tower. It is also known for its diverse culinary scene, night markets, and street food. The city is a hub for business, tourism, and culture, with a mix of modern skyscrapers and traditional architecture.'}

In [11]:
Review = 'Les ordinateurs portables GamersTech impressionne par ses performances exceptionnelles et son design élégant. De sa configuration matérielle robuste à un clavier RVB personnalisable et un système de refroidissement efficace, il établit un équilibre parfait entre prouesses de jeu et portabilité.'


prompt_template1= PromptTemplate.from_template('Translate the review in english {Review}')
chain1= LLMChain(llm=llm, prompt=prompt_template1,output_key='translate')
prompt_template2= PromptTemplate.from_template('Summerize the content in 2 senetnces {translate}')
chain2= LLMChain(llm=llm, prompt=prompt_template2,output_key='summerize')
prompt_template3= PromptTemplate.from_template('Give a title to the summerization {summerize}')
chain3= LLMChain(llm=llm, prompt=prompt_template3,output_key='title')


sschain=SequentialChain(chains=[chain1,chain2,chain3],
                        input_variables=["Review"],
                        output_variables=["translate","summerize","title"])

sschain.invoke(Review)


{'Review': 'Les ordinateurs portables GamersTech impressionne par ses performances exceptionnelles et son design élégant. De sa configuration matérielle robuste à un clavier RVB personnalisable et un système de refroidissement efficace, il établit un équilibre parfait entre prouesses de jeu et portabilité.',
 'translate': 'The GamersTech laptops impress with their exceptional performance and elegant design. From their robust hardware configuration to a customizable RGB keyboard and efficient cooling system, they strike a perfect balance between gaming prowess and portability.',
 'summerize': 'GamersTech laptops have outstanding performance and sleek design, featuring powerful hardware, customizable RGB keyboard, and effective cooling system for a perfect balance of gaming and portability.',
 'title': '"GamersTech Laptops: The Perfect Combination of Performance and Style"'}

### Chatmodels With ChatOpenAI


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [13]:
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment="gpt-35-turbo",
)

In [14]:
llm.invoke([SystemMessage(content="You are a AI doctor"),HumanMessage(content="I have a headache and running nose with throat inflammation and food poisoning")])

AIMessage(content="I'm sorry to hear that. Based on your symptoms, it sounds like you may be experiencing a viral or bacterial infection. It's important to drink plenty of fluids and get proper rest to help your body fight off the infection. Additionally, over-the-counter pain relievers like ibuprofen or acetaminophen can help relieve headache and throat pain. For food poisoning, it is important to avoid solid foods and stick to clear liquids like water, ginger ale, or electrolyte drinks until your symptoms improve. If your symptoms persist or worsen, it's important to seek medical attention from a healthcare provider.")

In [15]:
llm.invoke([SystemMessage(content="You are a social media influencer"),HumanMessage(content="how to address all the negtivity you face in social media comments")])

AIMessage(content="As a social media influencer, it's important to have a plan in place to address negativity in your comments. Here are some tips to help you deal with negative comments:\n\n1. Don't take it personally: Remember that negative comments are not a reflection of you as a person, but rather a reflection of the commenter's own issues or opinions. \n\n2. Respond professionally: If you do choose to respond, do so professionally and with empathy. Acknowledge the commenter's concerns and try to address them in a respectful manner.\n\n3. Ignore it: Sometimes it's best to simply ignore negative comments. Responding to every negative comment can be exhausting and may even encourage more negativity.\n\n4. Engage with your positive followers: Focus on engaging with your positive followers and building a community of supportive individuals who appreciate your content.\n\n5. Use moderation tools: Use the moderation tools available on social media platforms to filter out or block negati

### Prompt Template + LLM 


In [21]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [22]:
template='you are an assistant in translating the contents from english to Tamil'
human_template="{text}"

chatprompt= ChatPromptTemplate.from_messages([("system",template),("human",human_template)])

In [35]:
chain= chatprompt|llm
text="AI has been developing vastly across all the fields which also gives a solid foundation"
response=chain.invoke({"text":text})

In [39]:
response.content

'for the growth of the economy.\n\nஅணுகல் மொழியின் பல புலம்பலுக்கும் பல புலனாக மேம்படுகிறது எனது கருத்து. இது, பொருளின் வளர்ச்சிக்கு ஒரு மிகவும் நன்மைகளான அடிப்படையைக் கொடுக்கின்றது.'